In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash.dependencies
import plotly.io as pio
import math
import pysrt
from plotly.subplots import make_subplots
from plotly.graph_objs import *
from scipy.stats import spearmanr
import json

## Functions

In [2]:
def include_euler_angles(df):
    # Rotation Matrix Phone and Airpods
    mRotPhone = []
    mRotAir = []
    for index, row in df.iterrows():
        rotPhone_first = row[["phoneMotionData.rotationMatrix.m1.1","phoneMotionData.rotationMatrix.m1.2","phoneMotionData.rotationMatrix.m1.3",]]
        rotPhone_second = row[["phoneMotionData.rotationMatrix.m2.1","phoneMotionData.rotationMatrix.m2.2","phoneMotionData.rotationMatrix.m2.3"]]
        rotPhone_third =  row[["phoneMotionData.rotationMatrix.m3.1","phoneMotionData.rotationMatrix.m3.2","phoneMotionData.rotationMatrix.m3.3"]]
        rotPhone_nump = np.array([rotPhone_first.to_numpy(),rotPhone_second.to_numpy(),rotPhone_third.to_numpy()])
        mRotPhone.append(rotPhone_nump)

        rotAir_first = row[["airpodMotionData.rotationMatrix.m1.1","airpodMotionData.rotationMatrix.m1.2","airpodMotionData.rotationMatrix.m1.3",]]
        rotAir_second = row[["airpodMotionData.rotationMatrix.m2.1","airpodMotionData.rotationMatrix.m2.2","airpodMotionData.rotationMatrix.m2.3"]]
        rotAir_third =  row[["airpodMotionData.rotationMatrix.m3.1","airpodMotionData.rotationMatrix.m3.2","airpodMotionData.rotationMatrix.m3.3"]]
        rotAir_nump = np.array([rotAir_first.to_numpy(),rotAir_second.to_numpy(),rotAir_third.to_numpy()])
        mRotAir.append(rotAir_nump)

    # Relative Rotations
    mRotRelatives = []
    for index, rotPhone in enumerate(mRotPhone):
        mRotRelatives.append(rotPhone.dot(mRotAir[index].transpose()))

    # Yaw Values
    relativeYaws = []
    relativeRoll = []
    relativePitch = []
    for index, rotRelative in enumerate(mRotRelatives):
        relativeYaws.append(math.atan2(rotRelative[0,1],rotRelative[0,0])) #Indexes shifted because starting at 0
        relativeRoll.append(math.atan2(rotRelative[1,2],rotRelative[2,2])) #Indexes shifted because starting at 0
        relativePitch.append(- math.asin(rotRelative[0,2]))                #Indexes shifted because starting at 0

    #Convert Radians to Degree
    realtiveYawsDegree = np.degrees(relativeYaws)
    relativeRollDegree = np.degrees(relativeRoll)
    relativePitchDegree = np.degrees(relativePitch)
    
    #import into frame
    df["relativeYaw"] = realtiveYawsDegree.tolist()
    df["relativeRoll"] = relativeRollDegree.tolist()
    df["relativePitch"] = relativePitchDegree.tolist()

def read_data(url):
    with open(url,'r') as f:
        data = json.loads(f.read())
    
    #get infos and delete theme from json
    #infos = data.pop("infos")
    #df["motionTimestampDiff"] = df.apply(lambda x: x["airpodMotionData.timestamp"] - x["phoneMotionData.timestamp"],axis=1)

    # normalize json to readable frame
    df = pd.json_normalize(data,record_path=["timestamps"])

    # convert objects to floats
    df = df.apply(pd.to_numeric, errors='coerce')

    #set timestamp as index
    df.set_index('timestamp',inplace=True)
        
    #Velocity m/s to km/h
    if 'locationData.velocity' in df.columns:
        df["locationData.velocity"] = df["locationData.velocity"].apply(lambda x: x * 3.6)
    #calculate Headrotation
    df["headRotY"] = df.apply(lambda x: x["airpodMotionData.yaw"] - x["phoneMotionData.yaw"],axis=1)
    df["headRotY"] = np.degrees(df["headRotY"]).tolist()
    df["airpodsYawDegrees"] = np.degrees(df["airpodMotionData.yaw"]).tolist()
    df["phoneYawDegrees"] = np.degrees(df["phoneMotionData.yaw"]).tolist()
    
    df["airpodsRollDegrees"] = np.degrees(df["airpodMotionData.roll"]).tolist()
    df["phoneRollDegrees"] = np.degrees(df["phoneMotionData.roll"]).tolist()
    
    df["airpodsPitchDegrees"] = np.degrees(df["airpodMotionData.pitch"]).tolist()
    df["phonePitchDegrees"] = np.degrees(df["phoneMotionData.pitch"]).tolist()
    
    include_euler_angles(df)
    addMeanRotations(df)
    addRotations(df)
    
    return df
def read_srt(url):
    subs = pysrt.open(url)
    return subs
def add_conditions(subject,fig,propertyName): #To-Do use times instead of iterating srts again
    subs = srts[subject]
    shapes = []
    for sub in subs:
        # If Log was writte before video, add the delay to the times to get correct times
        if srt_delays[subject] < 0:
            start = sub.start.minutes * 60 + sub.start.seconds + abs(srt_delays[subject])
            end = sub.end.minutes * 60 + sub.end.seconds + abs(srt_delays[subject])
        else:
            start = sub.start.minutes * 60 + sub.start.seconds - abs(srt_delays[subject])
            end = sub.end.minutes * 60 + sub.end.seconds - abs(srt_delays[subject])
        #display("Condition lasted from {} to {}".format(start,end))#
        color = "#FFFFFF"
        if "Condition Start" in sub.text:
            color = COLOR_SIT_X_HEX
        elif "Condition X" in sub.text:
            color = COLOR_SIT_X_HEX
        elif "Condition A" in sub.text:
            color = COLOR_SIT_A_HEX
        elif "Condition B" in sub.text:
            color = COLOR_SIT_B_HEX
        elif "Condition C" in sub.text:
            color = COLOR_SIT_C_HEX
        elif "Condition D" in sub.text:
            color = COLOR_SIT_D_HEX
        elif "Condition E" in sub.text:
            color = COLOR_SIT_E_HEX
        else:
            color = COLOR_SIT_X_HEX
        shapes.append({
            'type': 'rect',
#            xref: 'x',
#            yref: 'paper',
            'x0': start,
            'y0': subjects[subject][propertyName].min(),
            'x1': end,
            'y1': subjects[subject][propertyName].max(),
            'fillcolor': color,
            'opacity': 0.5,
            'line': {
                'width': 0
            }
        })
    fig.update_layout(shapes = shapes)
def getTimes(subject):
    subs = srts[subject]
    situations = []
    iterations = [0,0,0,0,0]
    for sub in subs:
        # If Log was writte before video, add the delay to the times to get correct times
        if srt_delays[subject] < 0:
            start = sub.start.minutes * 60 + sub.start.seconds + abs(srt_delays[subject])
            end = sub.end.minutes * 60 + sub.end.seconds + abs(srt_delays[subject])
        else:
            start = sub.start.minutes * 60 + sub.start.seconds - abs(srt_delays[subject])
            end = sub.end.minutes * 60 + sub.end.seconds - abs(srt_delays[subject])
        #if "Condition Start" in sub.text:
        #elif "Condition X" in sub.text:
        #elif "Condition D" in sub.text:
        condition = "x"
        iteration = 0
        if "Condition A" in sub.text:
            iterations[0] += 1
            iteration = iterations[0]
            condition = "a"
        elif "Condition B" in sub.text:
            iterations[1] += 1
            iteration = iterations[1]
            condition = "b"
        elif "Condition C" in sub.text:
            iterations[2] += 1
            iteration = iterations[2]
            condition = "c"
        elif "Condition E" in sub.text:
            iterations[4] += 1
            iteration = iterations[4]
            condition = "e"
        else:
            continue
        subjects[subject].loc[((subjects[subject].index >= start) & (subjects[subject].index <= end)),"situation"] = condition
        result = subjects[subject].loc[((subjects[subject].index >= start) & (subjects[subject].index <= end))]
        situations.append((condition,iteration,result.index[0],result.index[-1]))
    return np.array(situations,dtype = ([('situation', 'U10'),('iteration', 'i4'), ('start_index', 'f4'),('end_index', 'f4')]))
def extractSrtGazes(subject):
    subs = gaze_srts[subject]
    gaze_count = [0,0]
    for sub in subs:
        # If Log was writte before video, add the delay to the times to get correct times
        if srt_delays[subject] < 0:
            start = sub.start.minutes * 60 + sub.start.seconds + abs(srt_delays[subject])
            end = sub.end.minutes * 60 + sub.end.seconds + abs(srt_delays[subject])
        else:
            start = sub.start.minutes * 60 + sub.start.seconds - abs(srt_delays[subject])
            end = sub.end.minutes * 60 + sub.end.seconds - abs(srt_delays[subject])
        if "Gaze Right" in sub.text:
            gaze_count[1] += 1
        elif "Gaze Left" in sub.text:
            gaze_count[0] += 1
        else:
            continue
        #subjects[subject].loc[((subjects[subject].index >= start) & (subjects[subject].index <= end)),"situation"] = condition
        #result = subjects[subject].loc[((subjects[subject].index >= start) & (subjects[subject].index <= end))]
    return gaze_count
def get_meanDuration(times,situation):
    durations = []
    for iteration in times[times["situation"] == situation]:
        durations.append(iteration["end_index"] - iteration["start_index"])
    return np.mean(durations)
def get_sdDuration(times,situation):
    durations = []
    for iteration in times[times["situation"] == situation]:
        durations.append(iteration["end_index"] - iteration["start_index"])
    return np.std(durations)
def splitDf(listElement,df,chunksize):
    df_1 = df.iloc[:chunksize,:]
    df_2 = df.iloc[chunksize:,:]
    listElement.append(df_1)
    return df_2

# Splits the df into chunks (size defined globally) and adds "mean_rotation" for the chunk,wich is the mean of the "relativeYaw"
def addMeanRotations(df):
    splittableDf = df
    chunks = []

    while len(splittableDf) > 0:
        splittableDf = splitDf(chunks,splittableDf,60)

    for chunk_df in chunks:
        df.loc[chunk_df.iloc[0].name:chunk_df.iloc[-1].name,"mean_rotation"] = np.mean(chunk_df["relativeYaw"])

# adds two rows to the dataframe. rotations which indicates the rotation for each row, group_index which indicates to which look count the row is sorted (1 group = 1 look)
def addRotations(df):

    df["rotation"] = "x"
    #Every value which is above the mean_rotation + threshhold is a right headrotation, below its a left headrotation.
    df.loc[(df["relativeYaw"] - df["mean_rotation"] > rotation_threshhold),"rotation"] = "r"
    df.loc[(df["relativeYaw"] - df["mean_rotation"] < rotation_threshhold),"rotation"] = "l"
    df['group_index'] = (df['rotation'] != df['rotation'].shift()).cumsum()
    
#Returns the rotations for each situation for given subject
def getRotations(subject,times):
    rotations = {
        "a" : [],
        "b" : [],
        "c" : [],
        "d" : [],
        "e" : []
    }    
    # For Each situaiton
    for key,values in rotations.items():
        # For each iteration of situation
        for split_sit in times[times["situation"] == key]:
            # group by rotation
            rot_groups = subjects[subject][split_sit["start_index"]:split_sit["end_index"]].groupby(['rotation'])["group_index"]
            # get unqiue groups to count amount
            rots = subjects[subject][split_sit["start_index"]:split_sit["end_index"]].groupby(['rotation'])["group_index"].nunique()
            values.append((rots.get("l" , 0),rots.get("r" , 0)))
    return rotations

def getRotationCount(df):
    # group by rotation
    rot_groups = df.groupby(['rotation'])["group_index"]
    # get unqiue groups to count amount
    rots = rot_groups.nunique()
    display(rots)
    return [rots.get("l" , 0),rots.get("r" , 0)]
def getAngles(subject,times):
    situation_angles = []
    # For every Situation...
    for key in ["a","b","c","d","e"]:
        #for every iteration of this situation
        for index,split_sit in enumerate(times[times["situation"] == key]):
            #group by rotations
            rot_groups = subjects[subject][split_sit["start_index"]:split_sit["end_index"]][["mean_rotation","rotation","relativeYaw","group_index"]]#.groupby(['rotation'])["group_index"]
            rot_groups.groupby(['rotation'])

            # for every found headrotation in iteration
            for group in rot_groups["group_index"].unique():
                df_group = rot_groups.loc[rot_groups["group_index"] == group]
                # left rotation
                if 'l' in df_group["rotation"].unique():
                    #get index of lowest angle in group
                    minIndex = df_group["relativeYaw"].idxmin()
                    #if (df_group.loc[minIndex]["relativeYaw"] - df_group.loc[minIndex]["mean_rotation"]) < -180:
                        #display("Found huge angle from {} with mean {}".format(df_group.loc[minIndex]["relativeYaw"],df_group.loc[minIndex]["mean_rotation"]))
                        #continue
                    situation_angles.append((key,index,"left",df_group.loc[minIndex]["relativeYaw"] - df_group.loc[minIndex]["mean_rotation"]))
                #right rotation
                elif 'r' in df_group["rotation"].unique():
                    #get index of highest angle in group
                    maxIndex = df_group["relativeYaw"].idxmax()
                    #calculate angle in relation to mean rotation
                    #if (df_group.loc[maxIndex]["relativeYaw"] - df_group.loc[maxIndex]["mean_rotation"]) > 180:
                        #display("Found huge angle from {} with mean {}".format(df_group.loc[maxIndex]["relativeYaw"],df_group.loc[maxIndex]["mean_rotation"]))
                        #continue
                    situation_angles.append((key,index,"right",df_group.loc[maxIndex]["relativeYaw"] - df_group.loc[maxIndex]["mean_rotation"]))
    return np.array(situation_angles,dtype = ([('situation', 'U10'),('iteration', 'i4'),('direction', 'U10'), ('angle', 'f4')]))

def getStandtimes(subject,times):
    situation_stands = []
    for situation in ["a","b","c","e"]:
        #for every iteration of this situation
        for index,split_sit in enumerate(times[times["situation"] == situation]):
            df_timeslice = subjects[subject].loc[split_sit["start_index"]:split_sit["end_index"]]["locationData.velocity"]
            group_index = (df_timeslice != df_timeslice.shift()).cumsum()
            # for every found headrotation
            standedInIteration = False
            for stand_sit_group in group_index.unique():
                df_sit = group_index.loc[group_index == stand_sit_group]
                if df_timeslice[df_sit.index[0]] == 0.0:
                    situation_stands.append((situation,index,df_sit.index[-1] - df_sit.index[0]))
                    standedInIteration = True
            if not standedInIteration:
                # This iteration no one stand. Adding 0 to account for in mean calculations
                situation_stands.append((situation,index,0.00))
    return np.array(situation_stands,dtype = ([('situation', 'U10'),('iteration', 'i4'), ('seconds', 'f4')]))
# builds sum over all iterations of one situation, returns mean of this sum
def getMeanOfStandSituation(times,situation):
    sit_times = times[times["situation"] == situation]
    sit_sums = []
    for iteration in np.unique(sit_times["iteration"]):
        sit_sums.append(np.sum(sit_times[sit_times["iteration"] == iteration]["seconds"]))
    return np.mean(sit_sums)
def getSdOfStandSituation(times,situation):
    sit_times = times[times["situation"] == situation]
    sit_sums = []
    for iteration in np.unique(sit_times["iteration"]):
        sit_sums.append(np.sum(sit_times[sit_times["iteration"] == iteration]["seconds"]))
    return np.std(sit_sums)

## Plots

In [3]:
# Headrotation interpretation Settings
# ChunkSize defines how the mean value is build to handle the drift in rotation data. 
# 120 would build the mean over 120 log entries which results in chunks of 60 seconds
chunkSize = 120
# Rotation threshhold defines which degree has to be reached to count as rotation. 
# Every rotation which overcomes this value will be counted as headrotation
rotation_threshhold = 20.0

#Subjects
df_subject1 = read_data("./logs/participants/2022-10-17 124155-logfile-subject-1.json")
df_subject2 = read_data("./logs/participants/2022-10-20 111557-logfile-subject-2.json")
df_subject3 = read_data("./logs/participants/2022-11-04 113121-logfile-subject-3.1.json")
df_subject4 = read_data("./logs/participants/2022-10-25 142229-logfile-subject-4.json")
df_subject5 = read_data("./logs/participants/2022-10-27 165339-logfile-subject-5.json")
df_subject6 = read_data("./logs/participants/2022-10-28 101829-logfile-subject-6.json")
df_subject7 = read_data("./logs/participants/2022-10-28 154837-logfile-subject-7.json")
df_subject8 = read_data("./logs/participants/2022-11-04 122345-logfile-subject-8.json")


# TO-DO: Streamline into one dictionary
subjects = {
    "Subject 1" : df_subject1,
    "Subject 2" : df_subject2,
    "Subject 3" : df_subject3,
    "Subject 4" : df_subject4,
    "Subject 5" : df_subject5,
    "Subject 6" : df_subject6,
    "Subject 7" : df_subject7,
    "Subject 8" : df_subject8
}
# Adding rotations where mean_rotation + threshhold as a identifier field to the dataframes
for df in subjects.values():
    addRotations(df)


rotation_threshhold = 25.0

#Subjects
df_subject1_25 = read_data("./logs/participants/2022-10-17 124155-logfile-subject-1.json")
df_subject2_25 = read_data("./logs/participants/2022-10-20 111557-logfile-subject-2.json")
df_subject3_25 = read_data("./logs/participants/2022-11-04 113121-logfile-subject-3.1.json")
df_subject4_25 = read_data("./logs/participants/2022-10-25 142229-logfile-subject-4.json")
df_subject5_25 = read_data("./logs/participants/2022-10-27 165339-logfile-subject-5.json")
df_subject6_25 = read_data("./logs/participants/2022-10-28 101829-logfile-subject-6.json")
df_subject7_25 = read_data("./logs/participants/2022-10-28 154837-logfile-subject-7.json")
df_subject8_25 = read_data("./logs/participants/2022-11-04 122345-logfile-subject-8.json")

subjects_25 = {
    "Subject 1" : df_subject1_25,
    "Subject 2" : df_subject2_25,
    "Subject 3" : df_subject3_25,
    "Subject 4" : df_subject4_25,
    "Subject 5" : df_subject5_25,
    "Subject 6" : df_subject6_25,
    "Subject 7" : df_subject7_25,
    "Subject 8" : df_subject8_25
}
for df in subjects_25.values():
    addRotations(df)


rotation_threshhold = 30.0

#Subjects
df_subject1_30 = read_data("./logs/participants/2022-10-17 124155-logfile-subject-1.json")
df_subject2_30 = read_data("./logs/participants/2022-10-20 111557-logfile-subject-2.json")
df_subject3_30 = read_data("./logs/participants/2022-11-04 113121-logfile-subject-3.1.json")
df_subject4_30 = read_data("./logs/participants/2022-10-25 142229-logfile-subject-4.json")
df_subject5_30 = read_data("./logs/participants/2022-10-27 165339-logfile-subject-5.json")
df_subject6_30 = read_data("./logs/participants/2022-10-28 101829-logfile-subject-6.json")
df_subject7_30 = read_data("./logs/participants/2022-10-28 154837-logfile-subject-7.json")
df_subject8_30 = read_data("./logs/participants/2022-11-04 122345-logfile-subject-8.json")

subjects_30 = {
    "Subject 1" : df_subject1_30,
    "Subject 2" : df_subject2_30,
    "Subject 3" : df_subject3_30,
    "Subject 4" : df_subject4_30,
    "Subject 5" : df_subject5_30,
    "Subject 6" : df_subject6_30,
    "Subject 7" : df_subject7_30,
    "Subject 8" : df_subject8_30
}
for df in subjects_30.values():
    addRotations(df)
    
#SRTs
srt_subject1 = read_srt("./logs/participants/srts/subs_participant_1.srt")
srt_subject2 = read_srt("./logs/participants/srts/subs_participant_2.srt")
srt_subject3 = read_srt("./logs/participants/srts/subs_participant_3.srt")
srt_subject4 = read_srt("./logs/participants/srts/subs_participant_4.srt")
srt_subject5 = read_srt("./logs/participants/srts/subs_participant_5.srt")
srt_subject6 = read_srt("./logs/participants/srts/subs_participant_6.srt")
srt_subject7 = read_srt("./logs/participants/srts/subs_participant_7.srt")
srt_subject8 = read_srt("./logs/participants/srts/subs_participant_8.srt")

#Gazes
gaze_srt_subject1 = read_srt("./logs/participants/srts/gazes/gazes_p1.srt")
gaze_srt_subject2 = read_srt("./logs/participants/srts/gazes/gazes_p2.srt")
gaze_srt_subject3 = read_srt("./logs/participants/srts/gazes/gazes_p3.srt")
gaze_srt_subject4 = read_srt("./logs/participants/srts/gazes/gazes_p4.srt")
gaze_srt_subject5 = read_srt("./logs/participants/srts/gazes/gazes_p5.srt")
gaze_srt_subject6 = read_srt("./logs/participants/srts/gazes/gazes_p6.srt")
gaze_srt_subject7 = read_srt("./logs/participants/srts/gazes/gazes_p7.srt")
gaze_srt_subject8 = read_srt("./logs/participants/srts/gazes/gazes_p8.srt")

In [4]:


srts = {
    "Subject 1" : srt_subject1,
    "Subject 2" : srt_subject2,
    "Subject 3" : srt_subject3,
    "Subject 4" : srt_subject4,
    "Subject 5" : srt_subject5,
    "Subject 6" : srt_subject6,
    "Subject 7" : srt_subject7,
    "Subject 8" : srt_subject8
}
gaze_srts = {
    "Subject 1" : gaze_srt_subject1,
    "Subject 2" : gaze_srt_subject2,
    "Subject 3" : gaze_srt_subject3,
    "Subject 4" : gaze_srt_subject4,
    "Subject 5" : gaze_srt_subject5,
    "Subject 6" : gaze_srt_subject6,
    "Subject 7" : gaze_srt_subject7,
    "Subject 8" : gaze_srt_subject8
}
# If negative, the app was running before the camera. Little inconsistency in the study design 
srt_delays = {
    "Subject 1" : - 8.0,
    "Subject 2" : - 29.0,
    "Subject 3" : 12.0 ,
    "Subject 4" : -10.0,
    "Subject 5" : 9.0,
    "Subject 6" : -32.0,
    "Subject 7" : 490.0,
    "Subject 8" : 14.0
}
#[difficulty,safety]
subjects_perceived = {
    "Subject 1" : [[5,1,3,1,1],[3,4,4,5,5]],
    "Subject 2" : [[2,1,1,1,1],[5,5,4,5,5]],
    "Subject 3" : [[1,1,4,3,2],[4,5,2,4,4]],
    "Subject 4" : [[2,3,3,3,3],[4,4,3,4,3]],
    "Subject 5" : [[1,4,3,1,2],[4,2,3,4,3]],
    "Subject 6" : [[1,1,2,3,2],[4,5,3,4,4]],
    "Subject 7" : [[2,1,3,1,1],[5,5,5,5,5]],
    "Subject 8" : [[1,3,2,4,1],[5,4,5,2,5]]
}
# Labels for the plots
available_data_labels = list(subjects.keys())

In [5]:
# Get mean values per subject, per situation
data = []
for key,value in subjects.items():
    real_gazes = extractSrtGazes(key)
    rotation_count = getRotationCount(value)
    data.append([
        key,
        value["relativeYaw"].mean(axis=0),
        value["locationData.velocity"].mean(axis=0),
        value.index[-1],
        real_gazes[0],
        real_gazes[1],
        rotation_count[0],
        rotation_count[1]
    ])
df_all = pd.DataFrame(data, columns=[
    'subject', 
    'mean_rotation',
    'mean_speed',
    "duration",
    "gaze_left_count", #real gazes, manually marked in video
    "gaze_right_count",
    "rotation_left_count", #measured rotations
    "rotation_right_count",
])
df_all.loc['Total'] = df_all.mean(numeric_only=True)
#df_all.loc['SD'] = df_all.std(numeric_only=True)



# Get mean values per subject, per situation
data = []
for key,value in subjects_25.items():
    real_gazes = extractSrtGazes(key)
    rotation_count = getRotationCount(value)
    data.append([
        key,
        value["relativeYaw"].mean(axis=0),
        value["locationData.velocity"].mean(axis=0),
        value.index[-1],
        real_gazes[0],
        real_gazes[1],
        rotation_count[0],
        rotation_count[1]
    ])
df_all_25 = pd.DataFrame(data, columns=[
    'subject', 
    'mean_rotation',
    'mean_speed',
    "duration",
    "gaze_left_count", #real gazes, manually marked in video
    "gaze_right_count",
    "rotation_left_count", #measured rotations
    "rotation_right_count",
])
df_all_25.loc['Total'] = df_all_25.mean(numeric_only=True)
#df_all.loc['SD'] = df_all.std(numeric_only=True)




# Get mean values per subject, per situation
data = []
for key,value in subjects_30.items():
    real_gazes = extractSrtGazes(key)
    rotation_count = getRotationCount(value)
    data.append([
        key,
        value["relativeYaw"].mean(axis=0),
        value["locationData.velocity"].mean(axis=0),
        value.index[-1],
        real_gazes[0],
        real_gazes[1],
        rotation_count[0],
        rotation_count[1]
    ])
df_all_30 = pd.DataFrame(data, columns=[
    'subject', 
    'mean_rotation',
    'mean_speed',
    "duration",
    "gaze_left_count", #real gazes, manually marked in video
    "gaze_right_count",
    "rotation_left_count", #measured rotations
    "rotation_right_count",
])
df_all_30.loc['Total'] = df_all_30.mean(numeric_only=True)
#df_all.loc['SD'] = df_all.std(numeric_only=True)

rotation
l    161
r    161
x      1
Name: group_index, dtype: int64

rotation
l    77
r    76
x     1
Name: group_index, dtype: int64

rotation
l    51
r    51
x     1
Name: group_index, dtype: int64

rotation
l    130
r    130
x      1
Name: group_index, dtype: int64

rotation
l    160
r    159
x      1
Name: group_index, dtype: int64

...

rotation
l    87
r    87
x     1
Name: group_index, dtype: int64

rotation
l    92
r    91
x     1
Name: group_index, dtype: int64

rotation
l    61
r    60
x     1
Name: group_index, dtype: int64

rotation
l    88
r    88
x     1
Name: group_index, dtype: int64

rotation
l    53
r    52
x     1
Name: group_index, dtype: int64

In [6]:
marking_red = "#961C26"

#"gaze_left_count", "gaze_right_count", "rotation_left_count", "rotation_right_count"
fig_duration = go.Figure()
fig_duration.add_trace(go.Bar(x=df_all.gaze_left_count, y=df_all.subject, 
                        orientation='h', showlegend=True,
                        text=np.round(df_all.gaze_left_count,decimals = 0),
                        name='Real Left Gazes', marker_color= marking_red))
fig_duration.add_trace(go.Bar(x=df_all.rotation_left_count, y=df_all.subject, 
                        orientation='h', showlegend=True,
                        text=np.round(df_all.rotation_left_count,decimals = 0),
                        name='Measured Left Rotations', marker_color='#000000'))
fig_duration.update_xaxes(showgrid=False)
fig_duration.update_yaxes(showgrid=False, categoryorder="category descending",
                 ticksuffix=' ', showline=False)
fig_duration.update_traces(hovertemplate=None)
fig_duration.update_layout(
#    title='Mean of Duration & Standingtimes per Situation (Seconds)(Field)',
    margin=dict(t=80, b=0, l=70, r=40),
    hovermode="y unified", 
    xaxis_title=' ', yaxis_title=" ",
    width=500, height=320,
    font=dict(family="Lato, sans-serif",size=14),
    legend=dict(orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5),
    hoverlabel=dict(font_size=13, font_family="Lato, sans-serif")
    #paper_bgcolor='rgba(0,0,0,0)',
    #plot_bgcolor='rgba(0,0,0,0)'
)

In [7]:
#"gaze_left_count", "gaze_right_count", "rotation_left_count", "rotation_right_count"
fig_duration = go.Figure()
fig_duration.add_trace(go.Bar(x=df_all.gaze_right_count, y=df_all.subject, 
                        orientation='h', showlegend=True,
                        text=np.round(df_all.gaze_right_count,decimals = 0),
                        name='Real Right Gazes', marker_color= marking_red))
fig_duration.add_trace(go.Bar(x=df_all.rotation_right_count, y=df_all.subject, 
                        orientation='h', showlegend=True,
                        text=np.round(df_all.rotation_right_count,decimals = 0),
                        name='Measured Right Rotations', marker_color='#000000'))
fig_duration.update_xaxes(showgrid=False)
fig_duration.update_yaxes(showgrid=False, categoryorder="category descending",
                 ticksuffix=' ', showline=False)
fig_duration.update_traces(hovertemplate=None)
fig_duration.update_layout(
#    title='Mean of Duration & Standingtimes per Situation (Seconds)(Field)',
    margin=dict(t=80, b=0, l=70, r=40),
    hovermode="y unified", 
    xaxis_title=' ', yaxis_title=" ",
    width=500, height=320,
    font=dict(family="Lato, sans-serif",size=14),
    legend=dict(orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5),
    hoverlabel=dict(font_size=13, font_family="Lato, sans-serif")
    #paper_bgcolor='rgba(0,0,0,0)',
    #plot_bgcolor='rgba(0,0,0,0)'
)

In [8]:
left_loss = df_all.loc["Total"].rotation_left_count / df_all.loc["Total"].gaze_left_count * 100
right_loss = df_all.loc["Total"].rotation_right_count / df_all.loc["Total"].gaze_right_count * 100

left_loss_25 = df_all_25.loc["Total"].rotation_left_count / df_all_25.loc["Total"].gaze_left_count * 100
right_loss_25 = df_all_25.loc["Total"].rotation_right_count / df_all_25.loc["Total"].gaze_right_count * 100

left_loss_30 = df_all_30.loc["Total"].rotation_left_count / df_all_30.loc["Total"].gaze_left_count * 100
right_loss_30 = df_all_30.loc["Total"].rotation_right_count / df_all_30.loc["Total"].gaze_right_count * 100

display("The Mean of real gazes to the left was {}, compared to {} measured gazes. Which gives us a {} % loss rate".format(df_all.loc["Total"].gaze_left_count,df_all.loc["Total"].rotation_left_count,left_loss))
display("The Mean of real gazes to the right was {}, compared to {} measured gazes. Which gives us a {} % loss rate".format(df_all.loc["Total"].gaze_right_count,df_all.loc["Total"].rotation_right_count,right_loss))

display("The Mean of real gazes to the left was {}, compared to {} measured gazes. Which gives us a {} % loss rate".format(df_all_25.loc["Total"].gaze_left_count,df_all_25.loc["Total"].rotation_left_count,left_loss_25))
display("The Mean of real gazes to the right was {}, compared to {} measured gazes. Which gives us a {} % loss rate".format(df_all_25.loc["Total"].gaze_right_count,df_all_25.loc["Total"].rotation_right_count,right_loss_25))

display("The Mean of real gazes to the left was {}, compared to {} measured gazes. Which gives us a {} % loss rate".format(df_all_30.loc["Total"].gaze_left_count,df_all_30.loc["Total"].rotation_left_count,left_loss_30))
display("The Mean of real gazes to the right was {}, compared to {} measured gazes. Which gives us a {} % loss rate".format(df_all_30.loc["Total"].gaze_right_count,df_all_30.loc["Total"].rotation_right_count,right_loss_30))

d = {'threshhold': [20,25,30], 'lossrate_left': [left_loss,left_loss_25,left_loss_30],'lossrate_right':[right_loss,right_loss_25,right_loss_30]}
df_lossrates = pd.DataFrame(data=d)

fig_duration = go.Figure()
fig_duration.add_trace(go.Bar(x=df_lossrates.threshhold, y=df_lossrates.lossrate_left, showlegend=True, text=np.round(df_lossrates.lossrate_left,decimals = 0), name='Lossrate Left', marker_color= marking_red))
fig_duration.add_trace(go.Bar(x=df_lossrates.threshhold, y=df_lossrates.lossrate_right, showlegend=True, text=np.round(df_lossrates.lossrate_right,decimals = 0), name='Lossrate Right', marker_color='#000000'))
fig_duration.update_xaxes(showgrid=False)
fig_duration.update_yaxes(showgrid=False, categoryorder="category descending",
                 ticksuffix=' ', showline=False)
fig_duration.update_traces(hovertemplate=None)
fig_duration.update_layout(
#    title='Mean of Duration & Standingtimes per Situation (Seconds)(Field)',
    margin=dict(t=80, b=0, l=70, r=40),
    hovermode="y unified", 
    xaxis_title='Threshhold', yaxis_title="Lossrate Percentage",
    font=dict(family="Lato, sans-serif",size=14),
    legend=dict(orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5),
    hoverlabel=dict(font_size=13, font_family="Lato, sans-serif")
    #paper_bgcolor='rgba(0,0,0,0)',
    #plot_bgcolor='rgba(0,0,0,0)'
)

'The Mean of real gazes to the left was 117.75, compared to 109.625 measured gazes. Which gives us a 93.09978768577494 % loss rate'

'The Mean of real gazes to the right was 83.125, compared to 109.25 measured gazes. Which gives us a 131.42857142857142 % loss rate'

'The Mean of real gazes to the left was 117.75, compared to 90.625 measured gazes. Which gives us a 76.96390658174099 % loss rate'

'The Mean of real gazes to the right was 83.125, compared to 90.0 measured gazes. Which gives us a 108.27067669172932 % loss rate'

'The Mean of real gazes to the left was 117.75, compared to 75.5 measured gazes. Which gives us a 64.11889596602973 % loss rate'

'The Mean of real gazes to the right was 83.125, compared to 74.75 measured gazes. Which gives us a 89.92481203007519 % loss rate'

In [9]:
means = [100,0,-100]
yaws = [[110,90,121,79,-20],[10,-10,21,-21,0],[-90,-110,-79,-121,20]]
thresh = 20

results = []
for mean,yaws in zip(means,yaws):
    result = []
    for yaw in yaws:
        if  yaw - mean > thresh:
            result.append("right")
        elif yaw - mean < - thresh:
            result.append("left")
        else:
            result.append("no value")
    results.append(result)
display(results)

[['no value', 'no value', 'right', 'left', 'left'],
 ['no value', 'no value', 'right', 'left', 'no value'],
 ['no value', 'no value', 'right', 'left', 'right']]

In [10]:
display(df_subject2.loc[df_subject2.rotation == "l"])



fig_duration = go.Figure()
fig_duration.add_trace(go.Scatter(mode="markers",x=df_subject2.index, y=df_subject2.rotation, showlegend=True, text=np.round(df_lossrates.lossrate_left,decimals = 0), name='Lossrate Left', marker_color= marking_red))
fig_duration.update_xaxes(showgrid=False)
fig_duration.update_yaxes(showgrid=False, categoryorder="category descending",
                 ticksuffix=' ', showline=False)
fig_duration.update_traces(hovertemplate=None)
fig_duration.update_layout(
#    title='Mean of Duration & Standingtimes per Situation (Seconds)(Field)',
    margin=dict(t=80, b=0, l=70, r=40),
    hovermode="y unified", 
    xaxis_title='Threshhold', yaxis_title="Lossrate Percentage",
    font=dict(family="Lato, sans-serif",size=14),
    legend=dict(orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5),
    hoverlabel=dict(font_size=13, font_family="Lato, sans-serif")
    #paper_bgcolor='rgba(0,0,0,0)',
    #plot_bgcolor='rgba(0,0,0,0)'
    )

,phoneBattery,airpodMotionData.timestamp,airpodMotionData.rotationRate.z,airpodMotionData.rotationRate.x,airpodMotionData.rotationRate.y,airpodMotionData.pitch,airpodMotionData.magneticField.y,airpodMotionData.magneticField.x,airpodMotionData.magneticField.z,airpodMotionData.magneticField.accuracy,...,airpodsRollDegrees,phoneRollDegrees,airpodsPitchDegrees,phonePitchDegrees,relativeYaw,relativeRoll,relativePitch,mean_rotation,rotation,group_index
timestamp,,,,,,,,,,,,,,,,,,,,,
1.165085,0.94,762007.623706,-0.002022,0.032764,-0.017743,-1.073767,0.0,0.0,0.0,-1.0,...,-4.534528,1.199794,-61.522306,16.531063,2.210962,78.174449,2.873266,4.994630,l,2
1.664911,0.94,762008.123239,-0.041935,-0.044114,-0.014333,-1.038477,0.0,0.0,0.0,-1.0,...,-4.473483,1.226002,-59.500331,16.519019,2.360732,76.145172,2.783257,4.994630,l,2
2.165612,0.94,762008.602247,-0.008850,0.019192,0.009143,-1.031680,0.0,0.0,0.0,-1.0,...,-4.631478,1.166847,-59.110896,16.526571,2.515869,75.769996,2.605604,4.994630,l,2
2.664987,0.94,762009.082347,0.017740,0.036596,0.001271,-1.043997,0.0,0.0,0.0,-1.0,...,-4.994832,1.189195,-59.816631,16.540248,2.764309,76.506042,2.509173,4.994630,l,2
3.165614,0.94,762009.627349,-0.003157,0.003358,0.007420,-1.042529,0.0,0.0,0.0,-1.0,...,-4.928547,1.195579,-59.732501,16.545982,2.747888,76.424971,2.493458,4.994630,l,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494.679138,0.87,763501.086972,0.028562,-0.142860,0.050065,-0.774176,0.0,0.0,0.0,-1.0,...,15.639708,-0.750607,-44.357028,18.484429,80.417742,14.259887,-47.051880,63.605501,l,154
1495.178408,0.87,763501.600987,0.173326,-0.224200,0.257463,-0.807444,0.0,0.0,0.0,-1.0,...,22.332218,-0.761616,-46.263157,18.433468,79.086041,11.476223,-50.619291,63.605501,l,154
1495.679116,0.87,763501.987144,-0.399359,0.131666,-0.536952,-0.832135,0.0,0.0,0.0,-1.0,...,25.156945,-0.685729,-47.677801,18.432843,80.050095,9.565400,-52.575041,63.605501,l,154
